# ДЗ:
1. Сделать разведочный анализ данных, найти пропуски и выбросы. Примеры есть в ноутбуке EDA_AUTO из 4 задания.
2. Заполнить пропуски. Заполнить медианой или обучить линейную регрессию и заполнить ею пропуски. Если заполните линейной регрессией пропуски, то с помощью таблицы корреляций найдите второй признак, который имеет сильную взаимосвязь с первым признаком. Создаете отдельный датафрейм из этих признаков, исключаете пропуски (не удаляете), обучаете линейну регрессию. Затем прогнозируете на исключенных данных с пропусками и заполняете эти пропуски предсказанным значением.
3. Обучить множественную линейную регрессию. Она будет являться base line, т.е. базовая модель. С ней вы будете сравнивать другие модели.
4. Используя кроссвалидацию, подбор гиперпараметров обучить модели Lasso regression, Ridge Regression, ElasticNet (модель где Ridge + Lasso вместе используется), DecisionTreeRegressor, SVR, RandomForestRegressor

In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV # Линейна регрессия, Ридж и Лассо
from sklearn.linear_model import LassoCV, RidgeCV, MultiTaskLassoCV # Ridge и Lasso Регрессия
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import make_scorer, r2_score

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor

import gc

sns.set()
%matplotlib inline

import sys
# np.set_printoptions(suppress=True)
# np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

DISPLAY_MAX_ROWS = 20 #20
pd.set_option('display.max_rows', DISPLAY_MAX_ROWS)
pd.set_option('display.max_column',None)
plt.style.use('seaborn-whitegrid')


# plt.rcParams["figure.figsize"] = (20, 15)

import warnings
warnings.filterwarnings('ignore')

In [54]:
df = pd.read_csv("data.csv")
df

,KS4_GPA5_Qtg.PV,KS4_GPA5_TC_T1.PV,KS4_GPA5_Nst.PV,KS4_GPA5_Nvd.PV,KS4_GPA5_Nnd.PV,KS4_GPA5_T4.PV,KS4_GPA5_Pk.PV
0,50.074285,50.656771,500.475537,1207.542773,912.161426,119.562970,51.248955
1,50.074926,50.656771,500.379883,1209.190723,914.203027,119.683643,51.255357
2,50.074836,50.666920,499.359570,1210.159082,914.924512,119.665576,51.263570
3,50.074957,50.666920,500.535840,1210.641602,915.881055,119.674750,51.267725
4,50.075145,50.666920,500.198145,1211.398438,916.435645,119.692188,51.264826
...,...,...,...,...,...,...,...
29707,50.070496,52.706500,459.827393,1238.176660,914.571973,128.579962,51.102349
29708,50.070217,52.716716,459.400806,1237.532422,913.476074,128.521655,51.102543
29709,50.070292,52.696284,459.942578,1237.783984,913.913086,128.503632,51.102639
29710,50.070389,52.675853,460.185791,1237.441895,913.895508,128.481744,51.103799


# Разведочный анализ, заполнение пропусков и устранение выбросов

In [55]:
df.describe()

,KS4_GPA5_Qtg.PV,KS4_GPA5_TC_T1.PV,KS4_GPA5_Nst.PV,KS4_GPA5_Nvd.PV,KS4_GPA5_Nnd.PV,KS4_GPA5_T4.PV,KS4_GPA5_Pk.PV
count,29712.000000,29712.000000,29712.000000,29712.000000,27711.000000,29712.000000,29712.000000
mean,50.075761,51.582940,496.375057,1228.936047,925.288586,123.147985,51.579620
std,0.004490,0.718957,16.656003,9.293723,12.649640,2.293297,58.430508
min,50.067757,50.118885,458.647290,1207.311621,910.000195,118.375299,51.074956
25%,50.072440,50.940938,484.903076,1222.372168,915.022510,121.428400,51.183080
50%,50.074476,51.725804,490.411133,1228.801709,920.266797,123.233508,51.244438
75%,50.079485,52.134428,509.792322,1236.431030,936.632568,125.019626,51.271976
max,50.089918,53.053829,526.688867,1264.442773,977.826953,128.815308,10123.000000


In [56]:
df['KS4_GPA5_Pk.PV'].sort_values(ascending=False)

20100    10123.000000
7611        51.402325
7613        51.401648
7623        51.401648
7633        51.401358
             ...     
29664       51.096454
29671       51.096261
29672       51.095778
29674       51.095585
28419       51.074956
Name: KS4_GPA5_Pk.PV, Length: 29712, dtype: float64

In [57]:
df.at[20100, 'KS4_GPA5_Pk.PV'] = df['KS4_GPA5_Pk.PV'].median()

In [58]:
print(df.isnull().any())
print(df.isna().any())

KS4_GPA5_Qtg.PV      False
KS4_GPA5_TC_T1.PV    False
KS4_GPA5_Nst.PV      False
KS4_GPA5_Nvd.PV      False
KS4_GPA5_Nnd.PV       True
KS4_GPA5_T4.PV       False
KS4_GPA5_Pk.PV       False
dtype: bool
KS4_GPA5_Qtg.PV      False
KS4_GPA5_TC_T1.PV    False
KS4_GPA5_Nst.PV      False
KS4_GPA5_Nvd.PV      False
KS4_GPA5_Nnd.PV       True
KS4_GPA5_T4.PV       False
KS4_GPA5_Pk.PV       False
dtype: bool


In [59]:
print(df.isnull().sum())
print(df.isna().sum())

KS4_GPA5_Qtg.PV         0
KS4_GPA5_TC_T1.PV       0
KS4_GPA5_Nst.PV         0
KS4_GPA5_Nvd.PV         0
KS4_GPA5_Nnd.PV      2001
KS4_GPA5_T4.PV          0
KS4_GPA5_Pk.PV          0
dtype: int64
KS4_GPA5_Qtg.PV         0
KS4_GPA5_TC_T1.PV       0
KS4_GPA5_Nst.PV         0
KS4_GPA5_Nvd.PV         0
KS4_GPA5_Nnd.PV      2001
KS4_GPA5_T4.PV          0
KS4_GPA5_Pk.PV          0
dtype: int64


In [60]:
df.fillna(df.mean(), inplace=True)

In [61]:
print(df.isnull().sum())

KS4_GPA5_Qtg.PV      0
KS4_GPA5_TC_T1.PV    0
KS4_GPA5_Nst.PV      0
KS4_GPA5_Nvd.PV      0
KS4_GPA5_Nnd.PV      0
KS4_GPA5_T4.PV       0
KS4_GPA5_Pk.PV       0
dtype: int64


# Linear Regression

In [62]:
inputs = df[['KS4_GPA5_Qtg.PV', 'KS4_GPA5_TC_T1.PV', 'KS4_GPA5_Nst.PV']] # предикторы
outputs = df[['KS4_GPA5_Nvd.PV', 'KS4_GPA5_Nnd.PV', 'KS4_GPA5_T4.PV', 'KS4_GPA5_Pk.PV']] # отклики, целевые признаки
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size = 0.2, random_state = 0)
model = LinearRegression()
model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')

Оценка точности модели: (0.8782805879005511, 0.8785388630322588)


# Lasso regression

In [63]:
alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10]

In [64]:
model = MultiTaskLassoCV(cv=5, random_state=42, alphas=alphas)
model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметра:", model.alpha_)

Оценка точности модели: (0.8782674161733935, 0.8784252212206272)
Наилучшее значение параметра: 0.0001


# Ridge Regression

In [65]:
model = RidgeCV(cv=5, alphas=alphas)
model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметра:", model.alpha_)

Оценка точности модели: (0.8782804382225461, 0.8785280035423259)
Наилучшее значение параметра: 0.0001


# ElasticNet

In [67]:
elastic_regressor = ElasticNet()
model = GridSearchCV(elastic_regressor, {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
    'l1_ratio': [0.01, 0.1, 0.5, 1],
}, cv=5)

model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.best_estimator_.score(X_train, y_train), model.best_estimator_.score(X_test, y_test)}')
print("Наилучшее значение параметров:", model.best_estimator_)

Оценка точности модели: (0.8740428971812804, 0.873719150150546)
Наилучшее значение параметров: ElasticNet(alpha=0.0001, l1_ratio=1)


# DecisionTreeRegressor

In [68]:
dec_tree_regressor = DecisionTreeRegressor()
model = GridSearchCV(dec_tree_regressor,param_grid={
    'max_features': [1, 2, 3]
})
model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметров: ", model.best_params_)

Оценка точности модели: (0.9999998625876905, 0.98505863576914)
Наилучшее значение параметров:  {'max_features': 3}


# SVR

In [69]:
model = GridSearchCV(MultiOutputRegressor(SVR()), param_grid={
    'estimator__gamma': [0.001, 0.01],
    'estimator__C': [0.001, 0.01],
}, n_jobs=-1)
model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметров: ", model.best_params_)


Оценка точности модели: (0.5286461118652093, 0.5238066990027468)
Наилучшее значение параметров:  {'estimator__C': 0.01, 'estimator__gamma': 0.01}


# RandomForestRegressor

In [70]:
model = GridSearchCV(RandomForestRegressor(), param_grid={
    'max_features': [1, 2, 3]
})
model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.best_estimator_.score(X_train, y_train), model.best_estimator_.score(X_test, y_test)}')
print("Наилучшее значение параметров: ", model.best_estimator_)

Оценка точности модели: (0.9988816651434768, 0.9912427981202456)
Наилучшее значение параметров:  RandomForestRegressor(max_features=2)


Лучшие модели: RandomForest (0.9988816651434768, 0.9912427981202456), DecisionTree (0.9999998625876905, 0.98505863576914)